<a href="https://colab.research.google.com/github/vibhahugar/vlsi/blob/main/MiniProject_Integrating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download required modules

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 23.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-col

#Download Flask#

In [ ]:
!pip install flask

Install pyngrok which will allow us to use a public url

In [ ]:
!pip install pyngrok

Make the direcytories


In [ ]:
import os

# Create necessary directories
os.makedirs('templates', exist_ok=True)
os.makedirs('static/css', exist_ok=True)
os.makedirs('static/js', exist_ok=True)

All Imports

In [ ]:
from flask import Flask, render_template, jsonify, request
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from pyngrok import ngrok
import re

Set the ngrok authtoken

In [ ]:
!ngrok authtoken 2hV4wsvza0ORnf8uP51ay8bc3Fl_3F9m1eg6LfPXRDB6dkBzk

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


All parameters being used

In [ ]:
################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

Function to load the model

In [ ]:
def load_model(model_name, use_4bit=True, bnb_4bit_compute_dtype='bfloat16', bnb_4bit_quant_type='nf4', use_nested_quant=True, device_map='auto'):
    # Determine the compute dtype
    compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

    # Configure Bits and Bytes
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=use_4bit,
        bnb_4bit_quant_type=bnb_4bit_quant_type,
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=use_nested_quant,
    )

    # Check GPU compatibility with bfloat16
    if compute_dtype == torch.float16 and use_4bit:
        major, _ = torch.cuda.get_device_capability()
        if major >= 8:
            print("=" * 80)
            print("Your GPU supports bfloat16: accelerate training with bf16=True")
            print("=" * 80)

    # Load base model with quantization configuration
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map=device_map
    )
    model.config.use_cache = False

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    return tokenizer, model


In [ ]:
model_name2 = "hidingbehindrainbows/Llama-2-7b-chat-finetune"
to, mo = load_model(model_name2)

config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

Function to generate and beautify the text

In [ ]:
def remove_text_between(string, start_marker, end_marker):
    # Create a regex pattern to match text between start and end markers
    pattern = re.escape(start_marker) + r'.*?' + re.escape(end_marker)

    # Use re.sub to replace the matched pattern with an empty string
    cleaned_string = re.sub(pattern, '', string)

    return cleaned_string


def generate_text(tokenizer, model, prompt, prevPrompts=None, max_length=200, max_new_tokens=50, num_return_sequences=1):
    combined_input = prompt

    logging.set_verbosity(logging.CRITICAL)

    # Create a text generation pipeline
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer,
                    max_length=max_length + len(combined_input), truncation=True, max_new_tokens=max_new_tokens)

    # Generate text with the model
    result = pipe(f"{combined_input}")
    result = result[0]['generated_text']
    response = result

    # Clean up the response using the remove_text_between function if needed
    response = remove_text_between(response, "<s>", "[/INST]")
    response = remove_text_between(response, response[0], "</s>")
    # response = remove_text_between(response, response[0], "[/INST]")
    response = remove_text_between(response, "[/INST]", "")
    response = remove_text_between(response, "[INST]", "")

    # remove stuff from previous stuff
    if prevPrompts:
      for i in prevPrompts:
        response.replace(i, "")

    # Dynamically truncate the text at the last sentence end punctuation if it exists
    last_sentence_end = max(response.rfind('.'), response.rfind('!'), response.rfind('?'))
    if last_sentence_end != -1:
        response = response[:last_sentence_end + 1]
    else:
        response = response[:max_length]

    return response


Functions to store converastion history

In [ ]:
conversation_history = []
def start_conversation(prompt):
    response = generate_text(to, mo, prompt)
    conversation_history.append({"user": prompt, "ai": response})
    return response

def continue_conversation(prompt):
    # Create a context from the most recent conversation segments
    context_segments = [f"<s> [INST]{seg['user']} [/INST] {seg['ai']} </s>" for seg in conversation_history[-2:]]
    context = " ".join(context_segments)
    response = generate_text(to, mo, f"{context} <s>[INST]{prompt}[/INST]", context_segments)
    return response

def run_conversation(prompt):

    if len(conversation_history) == 0:
        response = start_conversation(prompt)
    else:
        response = continue_conversation(prompt)

        # Update the conversation history with the latest user prompt and AI response
        conversation_history.append({"user": prompt, "ai": response})


    # Clear the conversation history periodically
    if len(conversation_history) % 3 == 0:
        conversation_history.clear()
        response += " Our conversation has unfortunately been deleted. Please provide more context for your next questions."

    return response



In [ ]:
while True:
  input_text = input("What is your prompt?\n")
  generated_text = run_conversation(input_text)
  print(generated_text)  # This print can be removed, it's just for checking the output

What is your prompt?
hi
 Hi! How are you?
What is your prompt?
who is america chavez
  America Chavez is a fictional character in the Marvel Comics universe. She first appeared in the comic book series "Invincible Iron Man" #13 in 2008.
What is your prompt?
who is her girlfriend
  America Chavez is a lesbian character, and her girlfriend is named Samantha "Sam" Chavez. Our conversation has unfortunately been deleted. Please provide more context for your next questions.
What is your prompt?
are you homophobic
are you homophobic?
 nobody is forcing you to be gay, and nobody is forcing you to be straight. you are free to be whoever you want to be.

and if you are homophobic, then you are not a good person.
What is your prompt?
do you think narendra modi is homophobic
 are you homophobic?
 nobody is forcing you to be gay, and nobody is forcing you to be straight. you are free to be whoever you want to be.

and  i don't think narendra modi is homophobic. he is a very good person, and he is 

KeyboardInterrupt: Interrupted by user

Connect to the public ngrok url

In [ ]:
public_url = ngrok.connect(5000)


#FLASK APP FUNCTIONS#

In [ ]:
# from google.colab.output import eval_js
# print(eval_js("google.colab.kernel.proxyPort(8000)"))

https://9y71mrpcs9g-496ff2e9c6d22116-8000-colab.googleusercontent.com/


In [ ]:
app = Flask(__name__, template_folder = "/content/templates")


@app.route("/")
def hello_world():
    return render_template("index.html")

@app.route("/api", methods=["GET", "POST"])
def qa():
    if request.method=="POST":
        print(request.json)
        question = request.json.get("question")
        generated_text = run_conversation(question)
        print(generated_text)
        data = {"result":f"{generated_text}"}
        return jsonify(data)

if __name__ == "__main__":
    public_url = ngrok.connect(5000)
    print(f"Public URL: {public_url}")

    # Run the Flask app
    app.run()

Public URL: NgrokTunnel: "https://281c-34-142-255-155.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
